In [1]:
import numpy as np
import pandas as pd

### DataFrame과 Series 간의 연산
- 다른 차원의 Numpy 배열과의 연산처럼 DataFrame과 Series 간의 연산도 잘 정의되어 있다.

In [2]:
# 2차원 배열과 그 배열의 한 로우의 차이점 알아보기

arr = np.arange(12).reshape((3,4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [3]:
arr[0]

array([0, 1, 2, 3])

In [4]:
arr - arr[0]

array([[0, 0, 0, 0],
       [4, 4, 4, 4],
       [8, 8, 8, 8]])

In [5]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns = list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])

Series = frame.iloc[0]

In [6]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [7]:
Series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

In [9]:
#기본적으로 DataFrame과 Series 간의 산술 연산은 Series의 색인을 DataFrame의 컬럼에 맞추고 아래 로우로 전파한다.
frame - Series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [11]:
# 만약 색인값을 DataFrame의 컬럼이나 Series의 색인에서 찾을 수 없다면 그 객체는 형식을 맞추기 위해 재색인된다.

series2 = pd.Series(range(3), index=['b','e','f'])

frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


In [12]:
# 각각 로운에 대해 연산하는 방법

series3 = frame['d']

frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [13]:
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [14]:
frame.sub(series3, axis='index')

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


### 함수 적용과 매핑
- pandas 객체에도 Numpy의 유니버셜 함수(배열의 각 원소에 적용되는 method)를 적용할 수 있다.

In [15]:
frame = pd.DataFrame(np.random.randn(4, 3), columns= list('bde'),
                    index=['Utah','Ohio','Texas','Oregon'])
frame

,b,d,e
Utah,3.612262,0.503188,1.388387
Ohio,0.332879,-0.791211,-0.241723
Texas,-0.151818,0.442488,-0.074785
Oregon,-0.353713,1.156286,-2.671384


In [16]:
np.abs(frame)

,b,d,e
Utah,3.612262,0.503188,1.388387
Ohio,0.332879,0.791211,0.241723
Texas,0.151818,0.442488,0.074785
Oregon,0.353713,1.156286,2.671384


In [19]:
# 자주 사용되는 또 다른 연산은 각 컬럼이나 로우의 1차원 배열에 함수를 적용하는 것이다.
# DataFrame의 apply method를 이용해 수행할 수 있다.

f = lambda X: X.max() - X.min()

frame.apply(f)


b    3.965975
d    1.947497
e    4.059771
dtype: float64

In [21]:
# 여기서 함수 f는 Series의 최댓값과 최솟값의 차이를 계산하는 함수다
# frame의 각 컬럼에 대해 한 번만 수행되며 결과값은 계산을 적용한 컬럼을 색인으로 하는 Series를 반환한다.
# apply 함수에 axis='columns' 인자를 넘기면 각 raw에 대해 한 번씩만 수행된다.

frame.apply(f, axis= 'columns')

Utah      3.109074
Ohio      1.124091
Texas     0.594306
Oregon    3.827670
dtype: float64

In [22]:
# 배열에 대한 일반적인 통계(sum or mean)는 DataFrmae의 method로 존재하므로 apply method를 사용할 필요 없다.
# apply method에 전달된 함수는 스칼라값을 반환할 필요가 없다. 여러 값을 가진 Series를 반환해도 된다.

def f(x):
    return pd.Series([x.min(), x.max()], index=['min','max'])

frame.apply(f)

,b,d,e
min,-0.353713,-0.791211,-2.671384
max,3.612262,1.156286,1.388387


In [24]:
# 배열의 각 원소에 적용되는 파이썬의 함수를 사용할 수도 있다.
# frame객체에서 실수값을 문자열 포맷으로 변환하고 싶다면 applymab을 이용해서 다음과 같이 할 수 있다.
format = lambda x: '%.2f' % x

frame.applymap(format)

,b,d,e
Utah,3.61,0.50,1.39
Ohio,0.33,-0.79,-0.24
Texas,-0.15,0.44,-0.07
Oregon,-0.35,1.16,-2.67


In [25]:
# method의 이름이 applymap인 이유는 Series는 가 원소에 적용할 함수를 지정하기 위한 map method를 가지고 있기 때문이다.

frame['e'].map(format)

Utah       1.39
Ohio      -0.24
Texas     -0.07
Oregon    -2.67
Name: e, dtype: object

 ### 정렬과 순위
 - 어떤 기준에 근거해서 데이터를 정렬하는 것 역시 중요한 명령이다.
 - raw나 column의 색인을 알파벳순으로 정렬하려면 정렬된 새로운 객체를 반환하는  sort_index method를 사용하면 된다.


In [26]:
obj = pd.Series(range(4), index=['d','a','b','c'])

obj

d    0
a    1
b    2
c    3
dtype: int64

In [27]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

In [30]:
# DataFrame은 raw나 column 중 하나의 축을 기준으로 정렬할 수 있다.
frame = pd.DataFrame(np.arange(8).reshape((2,4)),
                     index=['three','one'],
                     columns=['d','a','b','c'])

frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [31]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [34]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [35]:
# 데이터는 기본적으로 오름차수능로 정렬되고 내림차순으로 정렬할 수도 있다.

frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


In [37]:
# Series 객체를 값에 따라 정렬하고 싶다면 sort_values method를 사용하면 된다.

obj = pd.Series(np.random.randn(4))

obj

0    0.956085
1    0.227961
2   -0.326703
3   -0.012844
dtype: float64

In [38]:
obj.sort_values()

2   -0.326703
3   -0.012844
1    0.227961
0    0.956085
dtype: float64

In [39]:
# 정렬할 때 비어 있는 값은 기본적으로 Series 객체에서 가장 마지막에 위치한다.

obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])

obj

0    4.0
1    NaN
2    7.0
3    NaN
4   -3.0
5    2.0
dtype: float64

In [40]:
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

In [41]:
# DataFrame에서 하나 이상의 column에 있는 값으로 정렬을 하는 경우
# sort_values 함수의 by 옵션에 하나 이상의 column 이름을 남겨야한다.

frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, -1, 0, 1]})

frame

,b,a
0,4,0
1,7,-1
2,-3,0
3,2,1


In [44]:
frame.sort_values(by='b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,-1


In [45]:
frame.sort_values(by=['a','b'])

,b,a
1,7,-1
2,-3,0
0,4,0
3,2,1


In [46]:
# 순위는 정렬과 거의 흡사한데, 1부터 배열의 유효한 데이터 개수까지 순서를 매긴다.
# 기본적으로 Series와 DataFrame의 rank method는 동점인 항목에 대해서는 평균 순위를 매긴다.
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])

obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

#### 데이터 상에서 나타나는 순서에 따라 순위를 매기는 방법

In [47]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

In [50]:
# 여기서 0번재와 2번째 항목에 대해 평균 순위인 6.5를 적용하는 대신 먼저 출현한 순서대로 6과 7을 적용했다.
# 내림차순 적용 방법

print(obj)

print(obj.rank(ascending = False, method='max'))

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
dtype: int64
0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64


In [51]:
# DataFrame에서는  raw와 column에 대해 순위를 정할 수 있다.

frame = pd.DataFrame({'b':np.random.randn(4), 'a':np.random.randn(4),
                     'c':np.random.randn(4)})

frame

,b,a,c
0,-0.340471,-0.841859,-0.095253
1,1.568420,-0.911834,1.048618
2,-0.189280,-0.918777,-0.109000
3,-0.179425,-0.715236,-1.430425


In [52]:
frame.rank(axis='columns')

,b,a,c
0,2.0,1.0,3.0
1,3.0,1.0,2.0
2,2.0,1.0,3.0
3,3.0,2.0,1.0
